<a href="https://colab.research.google.com/github/YoonHyeJu/project1/blob/master/Gradio_sensitive_and_IdCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/grive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/grive


In [3]:
cd /content/grive/My Drive/Colab Notebooks

/content/grive/My Drive/Colab Notebooks


In [4]:
!pip install gradio 

     |████████████████████████████████| 983kB 8.9MB/s 
     |████████████████████████████████| 215kB 31.1MB/s 
     |████████████████████████████████| 2.7MB 31.7MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 962kB 52.8MB/s 


In [5]:
!pip install doqu

     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for doqu: filename=doqu-0.28.2-cp36-none-any.whl size=80917 sha256=1d5f6868d571c96e626416e012438ff0f18ac65a37366caf13e5b39be068a7e4
  Stored in directory: /root/.cache/pip/wheels/e3/3b/0a/773eefceefaf461b52b68a846e4de8c0030966e11319030721
Successfully built doqu


In [6]:
!pip install utils

In [9]:
cd /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3

/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3


In [10]:
import argparse

In [11]:
from models import * # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
import cv2

In [61]:
def ID_card_detect(source):
    inp = Image.fromarray(source.astype('uint8'), 'RGB')
    filename = "test.jpeg"
    inp.save(filename, "JPEG")

    cfg = '/content/grive/My Drive/Colab Notebooks/ID_card_detection/yolov3/cfg/yolov3-tiny.cfg'
    names = '/content/grive/My Drive/Colab Notebooks/ID_card_detection/yolov3/data/coco_id.names'
    weights = '/content/grive/My Drive/Colab Notebooks/ID_card_detection/yolov3/weights/ID_Card_Detection.pt'
    out = 'output'
    img_size=512
    conf_thres=0.3
    iou_thres=0.6
    fourcc='mp4v'
    half=False
    device=''
    view_img=False
    save_txt = False
    classes=None
    agnostic_nms=False
    augment=False
    cfg = check_file(cfg)
    names = check_file(names)
    save_img = True


    imgsz = (320, 192) if ONNX_EXPORT else img_size

    device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else device)
    if os.path.exists(out):
        shutil.rmtree(out)
    os.makedirs(out)

    model = Darknet(cfg, imgsz)

    attempt_download(weights)
    if weights.endswith('.pt'):
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:
        load_darknet_weights(model, weights)

    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Eval mode
    model.to(device).eval()

    # Fuse Conv2d + BatchNorm2d layers
    # model.fuse()
    # Export mode
    if ONNX_EXPORT:
        model.fuse()
        img = torch.zeros((1, 3) + imgsz)  # (1, 3, 320, 192)
        f = weights.replace(weights.split('.')[-1], 'onnx')  # *.onnx filename
        torch.onnx.export(model, img, f, verbose=False, opset_version=11,
                          input_names=['images'], output_names=['classes', 'boxes'])

        # Validate exported model
        import onnx
        model = onnx.load(f)  # Load the ONNX model
        onnx.checker.check_model(model)  # Check that the IR is well formed
        print(onnx.helper.printable_graph(model.graph))  # Print a human readable representation of the graph
        return

    # Half precision
    half = half and device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()

    # Set Dataloader
    vid_path, vid_writer = None, None
    #############
    dataset = LoadImages(filename, img_size=imgsz)
    names=load_classes(names)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img.float()) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=augment)[0]
        t2 = torch_utils.time_synchronized()

        # to float
        if half:
            pred = pred.float()

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres,
                                   multi_label=False, classes=classes, agnostic=agnostic_nms)
        
        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections for image i
            p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from imgsz to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].detach().unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)
    if save_txt or save_img:
        print('Results saved to %s' % os.getcwd() + os.sep + out)
        if platform == 'darwin':  # MacOS
            os.system('open ' + save_path)

    print('Done. (%.3fs)' % (time.time() - t0))
    print('save path: ')
    out_path=os.getcwd() + os.sep + save_path
    return out_path


In [62]:
def sensitive_label_detect(source):
    inp = Image.fromarray(source.astype('uint8'), 'RGB')
    filename = "test.jpeg"
    inp.save(filename, "JPEG")

    cfg = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/cfg/yolov3-tiny.cfg'
    names = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/data/coco_id.names'
    weights = '/content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/weights/last.pt'
    out = 'output'
    img_size=512
    conf_thres=0.3
    iou_thres=0.6
    fourcc='mp4v'
    half=False
    device=''
    view_img=False
    save_txt = False
    classes=None
    agnostic_nms=False
    augment=False
    cfg = check_file(cfg)
    names = check_file(names)
    save_img = True


    imgsz = (320, 192) if ONNX_EXPORT else img_size

    device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else device)
    if os.path.exists(out):
        shutil.rmtree(out)
    os.makedirs(out)

    model = Darknet(cfg, imgsz)

    attempt_download(weights)
    if weights.endswith('.pt'):
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:
        load_darknet_weights(model, weights)

    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Eval mode
    model.to(device).eval()
    
    # Fuse Conv2d + BatchNorm2d layers
    # model.fuse()

    # Export mode
    if ONNX_EXPORT:
        model.fuse()
        img = torch.zeros((1, 3) + imgsz)  # (1, 3, 320, 192)
        f = weights.replace(weights.split('.')[-1], 'onnx')  # *.onnx filename
        torch.onnx.export(model, img, f, verbose=False, opset_version=11,
                          input_names=['images'], output_names=['classes', 'boxes'])

        # Validate exported model
        import onnx
        model = onnx.load(f)  # Load the ONNX model
        onnx.checker.check_model(model)  # Check that the IR is well formed
        print(onnx.helper.printable_graph(model.graph))  # Print a human readable representation of the graph
        return

    # Half precision
    half = half and device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()

    # Set Dataloader
    vid_path, vid_writer = None, None
    #############
    dataset = LoadImages(filename, img_size=imgsz)
    names=load_classes(names)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img.float()) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=augment)[0]
        t2 = torch_utils.time_synchronized()

        # to float
        if half:
            pred = pred.float()

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres,
                                   multi_label=False, classes=classes, agnostic=agnostic_nms)
        
        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections for image i
            p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from imgsz to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].detach().unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        print('Results saved to %s' % os.getcwd() + os.sep + out)
        if platform == 'darwin':  # MacOS
            os.system('open ' + save_path)

    print('Done. (%.3fs)' % (time.time() - t0))
    print('save path: ')
    out_path=os.getcwd() + os.sep + save_path
    return out_path

In [65]:
def FN(img,choice):
    if choice ==  "Sensitive Label Detection":
        #!cd /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3
        return sensitive_label_detect(img)
    elif choice ==  "ID Card Detection":
        !cd /content/grive/My Drive/Colab Notebooks/ID_card_detection/yolov3
        import models
        import utils
        return ID_card_detect(img)

In [66]:
image = gr.inputs.Image()
radio = gr.inputs.Radio(["Sensitive Label Detection", "ID Card Detection"])

label = gr.outputs.Image(label = 'Result', type='file')
#checked = gr.outputs.text()

gr.Interface(fn= FN, inputs=[image,radio], outputs=label, capture_session=True).launch(debug=True) 
# if radio == "Sensitive Label Detection":
#     FN = sensitive_label_detect
# elif radio == "ID Card Detection":
#     FN = ID_card_detect


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on External URL: https://24629.gradio.app
Interface loading below...


/bin/bash: line 0: cd: too many arguments
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 37 layers, 8.66988e+06 parameters, 8.66988e+06 gradients
image 1/1 test.jpeg: 512x320 1 ID_cards, Done. (0.004s)
Results saved to /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/output
Done. (0.087s)
save path: 
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 37 layers, 8.07261e+06 parameters, 8.07261e+06 gradients
image 1/1 test.jpeg: 512x384 3 sensitive_labels, Done. (0.004s)
Results saved to /content/grive/My Drive/Colab Notebooks/sensitive_label_detection/yolov3/output
Done. (0.085s)
save path: 
/bin/bash: line 0: cd: too many arguments
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 37 layers, 8.66988e+06 parameters, 8.66988e+06 gradients
image 1/1 test.jpeg: 512x320 Done. (0.005s)
Results s

KeyboardInterrupt: ignored